#Import Required Libraries

In [2]:
import numpy as np
import pandas as pd
import time

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.metrics import accuracy_score


#Load Dataset

In [3]:
from sklearn.datasets import load_wine

# 1. Replace the data loading section
data = load_wine()
X = data.data
y = data.target
feature_names = data.feature_names

# Everything else (Splitting, Chi-Square, PCA, etc.) remains the same!


#Train–Test Split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Feature Scaling (Important for SVM)

In [5]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


#GRID SEARCH

In [6]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}


In [7]:
svm = SVC()

start_time = time.time()

grid_search = GridSearchCV(svm,param_grid,cv=5,scoring='accuracy')

grid_search.fit(X_train_scaled, y_train)

grid_time = time.time() - start_time


In [8]:
best_grid_model = grid_search.best_estimator_

y_pred_grid = best_grid_model.predict(X_test_scaled)
grid_accuracy = accuracy_score(y_test, y_pred_grid)

print("Grid Search Best Parameters:", grid_search.best_params_)
print("Grid Search Accuracy:", grid_accuracy)
print("Grid Search Time:", grid_time)


Grid Search Best Parameters: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Grid Search Accuracy: 0.9722222222222222
Grid Search Time: 0.7947211265563965


#RANDOM SEARCH

In [9]:
param_random = {
    'C': np.logspace(-2, 2, 10),
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}


In [10]:
start_time = time.time()

random_search = RandomizedSearchCV(
    svm,
    param_random,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    random_state=42
)

random_search.fit(X_train_scaled, y_train)

random_time = time.time() - start_time


In [11]:
best_random_model = random_search.best_estimator_

y_pred_random = best_random_model.predict(X_test_scaled)
random_accuracy = accuracy_score(y_test, y_pred_random)

print("Random Search Best Parameters:", random_search.best_params_)
print("Random Search Accuracy:", random_accuracy)
print("Random Search Time:", random_time)


Random Search Best Parameters: {'kernel': 'linear', 'gamma': 'scale', 'C': np.float64(0.027825594022071243)}
Random Search Accuracy: 1.0
Random Search Time: 0.17379450798034668


In [12]:
comparison_df = pd.DataFrame({
    "Method": ["Grid Search", "Random Search"],
    "Accuracy": [grid_accuracy, random_accuracy],
    "Computation Time (seconds)": [grid_time, random_time]
})

print(comparison_df)


          Method  Accuracy  Computation Time (seconds)
0    Grid Search  0.972222                    0.794721
1  Random Search  1.000000                    0.173795


Accuracy-wise: Grid Search

Time-wise: Random Search

Best Trade-off: Random Search for large datasets

In this experiment, an SVM classifier was trained using both Grid Search and Random Search for hyperparameter tuning. Grid Search achieved slightly higher accuracy by exhaustively exploring all parameter combinations, but it required significantly more computation time. Random Search provided competitive accuracy in much less time by sampling a limited number of parameter combinations. Therefore, while Grid Search is suitable when accuracy is the top priority, Random Search is more efficient for large datasets or limited computational resources.